In [ ]:
!pip install -q --upgrade transformers datasets accelerate arabert gradio


In [ ]:
import pandas as pd
import numpy as np
import random
import torch

from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

from arabert.preprocess import ArabertPreprocessor
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

الخلية 3 – تجهيز الداتا + خلط الأعمدة الثلاثة

In [1]:

CSV_PATH = "/content/merged_dataset_clean2.csv"

# Columns
HUMAN_COL = "human_collected_dataset"
AI_COLS = ["gemini_rephrased_v2_5", "rephrased_v_llm", "rewritten"]

# قراءة الملف
df = pd.read_csv(CSV_PATH, encoding="utf-8-sig")
print("# of rows:", len(df))


for col in [HUMAN_COL] + AI_COLS:
    if col not in df.columns:
        raise ValueError(f"❌ العمود {col} غير موجود في الملف!")

df = df[df[HUMAN_COL].astype(str).str.strip() != ""].copy()

def is_non_empty_text(x):
    return isinstance(x, str) and x.strip() != ""

# Cleaning rows with empty Ai
mask_has_any_ai = df[AI_COLS].apply(
    lambda row: any(is_non_empty_text(v) for v in row),
    axis=1
)
df = df[mask_has_any_ai].copy()
df.reset_index(drop=True, inplace=True)

print("عدد الصفوف بعد إزالة الصفوف بدون AI:", len(df))

# Mixing columns
def pick_random_ai(row):
    candidates = []
    for col in AI_COLS:
        val = row[col]
        if is_non_empty_text(val):
            candidates.append(val)
    if not candidates:
        return np.nan
    return random.choice(candidates)

df["ai_mixed"] = df.apply(pick_random_ai, axis=1)

# Removing empty rows (if any)
df = df[df["ai_mixed"].astype(str).str.strip() != ""].copy()
df.reset_index(drop=True, inplace=True)

print("final mixed ai rows: ", len(df))


human_df = pd.DataFrame({
    "text": df[HUMAN_COL].astype(str),
    "label": 0 # human
})

ai_df = pd.DataFrame({
    "text": df["ai_mixed"].astype(str),
    "label": 1 # Ai
})

full_df = pd.concat([human_df, ai_df], ignore_index=True)
full_df = full_df.sample(frac=1.0, random_state=SEED).reset_index(drop=True)

print("\n توزيع الليبلات بعد الدمج:")
print(full_df["label"].value_counts())
full_df.head()


NameError: name 'pd' is not defined

الخلية 4 – تقسيم 70/20/10 إلى Train / Val / Test

In [ ]:
# 70% تدريب، 20% Val، 10% Test مع stratify للحفاظ على التوازن
train_df, temp_df = train_test_split(
    full_df,
    test_size=0.3,
    stratify=full_df["label"],
    random_state=SEED,
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=1/3,  # ثلث الـ30% ≈ 10% من الكل
    stratify=temp_df["label"],
    random_state=SEED,
)

print("Train size:", len(train_df))
print("Val size  :", len(val_df))
print("Test size :", len(test_df))

# تحويل إلى HuggingFace Datasets
train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
val_ds   = Dataset.from_pandas(val_df.reset_index(drop=True))
test_ds  = Dataset.from_pandas(test_df.reset_index(drop=True))

raw_datasets = DatasetDict({
    "train": train_ds,
    "validation": val_ds,
    "test": test_ds
})
raw_datasets


Train size: 10614
Val size  : 3033
Test size : 1517


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10614
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 3033
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1517
    })
})

 الخلية 5 – إعداد AraBERT + Tokenizer + Preprocessor

In [ ]:
MODEL_NAME = "aubmindlab/bert-base-arabertv02"  # AraBERT v2 base

arabert_prep = ArabertPreprocessor(model_name=MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_batch(examples):
    texts = [arabert_prep.preprocess(t) for t in examples["text"]]
    tokenized = tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=256, # Changed from 128 --> 256.
    )
    tokenized["labels"] = examples["label"]
    return tokenized

tokenized_datasets = raw_datasets.map(
    preprocess_batch,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

train_dataset = tokenized_datasets["train"]
val_dataset   = tokenized_datasets["validation"]
test_dataset  = tokenized_datasets["test"]

train_dataset[0]


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/10614 [00:00<?, ? examples/s]

Map:   0%|          | 0/3033 [00:00<?, ? examples/s]

Map:   0%|          | 0/1517 [00:00<?, ? examples/s]

{'input_ids': [2,
  1266,
  1992,
  4661,
  16733,
  10278,
  1252,
  1115,
  34449,
  5787,
  2245,
  23,
  139,
  30,
  20,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

الخلية 6 – تحميل المودل وتعريف الـmetrics

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 2  # 0 = human, 1 = ai_generated

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels
).to(device)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary", zero_division=0
    )

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }


model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


الخلية 7 – إعداد TrainingArguments + Trainer

In [ ]:
from transformers import TrainingArguments, Trainer

batch_size = 16

training_args = TrainingArguments(
    output_dir="/content/arabert_ai_detector",
    learning_rate=3e-5, # increased LR
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=6, # was 3 -> changed to 6
    weight_decay=0.01,
    logging_dir="/content/logs",
    logging_steps=50,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
trainer


/tmp/ipython-input-3616478987.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
import torch
if torch.cuda.is_available():
    print(f"✅ GPU is available: {torch.cuda.get_device_name(0)}")
else:
    print("❌ GPU is NOT available. Please change runtime type!")

✅ GPU is available: Tesla T4


الخلية 8 – التدريب

In [ ]:
trainer.train()


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohammedahalomar (mohammedahalomar-king-saud-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,0.556500
100,0.447700
150,0.411200
200,0.377900
250,0.364500
300,0.386100
350,0.357200
400,0.409500
450,0.356600
500,0.307700


TrainOutput(global_step=3984, training_loss=0.24137615373575064, metrics={'train_runtime': 3142.5001, 'train_samples_per_second': 20.265, 'train_steps_per_second': 1.268, 'total_flos': 8377982224773120.0, 'train_loss': 0.24137615373575064, 'epoch': 6.0})

الخلية 9 – التقييم + حفظ المودل

In [ ]:
print("\n===== Validation =====")
val_metrics = trainer.evaluate(eval_dataset=val_dataset)
print(val_metrics)

print("\n===== Test =====")
test_predictions = trainer.predict(test_dataset)
y_true = test_predictions.label_ids
y_pred = np.argmax(test_predictions.predictions, axis=-1)

print(classification_report(
    y_true,
    y_pred,
    target_names=["human", "ai_generated"],
    digits=4
))



===== تقييم على مجموعة الـValidation =====


{'eval_loss': 0.6684088706970215, 'eval_accuracy': 0.8387734915924827, 'eval_precision': 0.8372703412073491, 'eval_recall': 0.8411338167435728, 'eval_f1': 0.839197632357777, 'eval_runtime': 40.2296, 'eval_samples_per_second': 75.392, 'eval_steps_per_second': 4.723, 'epoch': 6.0}

===== تقييم على مجموعة الـTest =====
              precision    recall  f1-score   support

       human     0.8552    0.8485    0.8519       759
ai_generated     0.8495    0.8562    0.8528       758

    accuracy                         0.8523      1517
   macro avg     0.8524    0.8523    0.8523      1517
weighted avg     0.8524    0.8523    0.8523      1517



In [ ]:
save_path = "/content/arabert_ai_detector_final"
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

print(f"\n✅ Saving Model & Tokenizer in: {save_path}")



✅ تم حفظ المودل والـtokenizer في: /content/arabert_ai_detector_final


In [ ]:
import gradio as gr
import torch.nn.functional as F

# نضمن إننا نستخدم نفس الـpreprocessor و tokenizer و model المتدربين
model.eval()

label_map = {
    0: "Human",
    1: "AI Generated"
}

def predict_text(text):
    if not isinstance(text, str) or text.strip() == "":
        return "أدخل نص أولاً", 0.0, 0.0

    # تحضير النص
    preprocessed = arabert_prep.preprocess(text)
    inputs = tokenizer(
        preprocessed,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=256,
    )

    # نقل إلى الجهاز (CPU/GPU)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=-1).cpu().numpy()[0]

    pred_label = int(np.argmax(probs))
    human_prob = float(probs[0])
    ai_prob = float(probs[1])

    result_label = label_map[pred_label]
    return result_label, human_prob, ai_prob

demo = gr.Interface(
    fn=predict_text,
    inputs=gr.Textbox(lines=4, label="أدخل النص العربي هنا"),
    outputs=[
        gr.Label(label="التصنيف"),
        gr.Number(label="احتمال أنه بشري (Human)"),
        gr.Number(label="احتمال أنه AI Generated"),
    ],
    title="Arabic AI Text Detector (AraBERT)",
    description="مودل يفرّق بين النص العربي البشري والنص العربي المولّد بالذكاء الاصطناعي."
)

demo.launch(share=True)


NameError: name 'model' is not defined


**Accuracy**: 0.7902298850574713

**F1-score** : 0.8232445520581114

**Precision:** 0.7112970711297071

**Recall**   : 0.9770114942528736